In [1]:
import pandas as pd

In [2]:
maxrg=250 # maxrg → 250
q='a+restrepo,d'
df=pd.read_json('http://inspirehep.net/search?p={}&of=recjson&rg={}'.format(q,maxrg))

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)

In [4]:
# Extract institute=arXiv from system_control_number insitutes list
ps=df.system_control_number.apply(lambda x: [x] if type(x)==dict else x).apply(
                               lambda x: [y for y in x if y.get('institute')=='arXiv'] ).str[0]
# Fill nan with empty dictionary
df['arXiv']=ps.apply(lambda x: {} if pd.isnull(x) else x).apply(
         lambda x: x.get('value') ).fillna('').str.split(':').str[-1]

df['publication_info']=df['publication_info'].apply(lambda x: x[0] if type(x)==list else x).apply(
                          lambda x: {} if pd.isnull(x) else x)

In [5]:
pi=pd.DataFrame( list(df.publication_info.values) )

pi=pi.rename({'title':'journal'},axis='columns')

pi['year']=pi.year.fillna('-1').astype(int)

pi['doi']=df.doi.apply(lambda x: x[0] if type(x)==list else x).fillna('')

pi['authors']=df.authors.apply( lambda x: [ y.get('full_name') for y in x] )

pi['title']=df.title.apply(lambda x: x.get('title'))

pi['pages']=df.physical_description.apply(lambda x: x.get('pagination'))

pi['imprint']=df.imprint.apply(lambda x: {} if pd.isnull(x) else x).apply(
                 lambda x: x.get('date')).combine(
    pi.year,func=lambda x,y: y if y>-1 and pd.isnull(x) else x).fillna(
                            '0000-00-00').astype(str).apply(
                 lambda x: x+'-00-00' if len( x.split('-'))==1 else x).apply(
                 lambda x: x+'-00' if len( x.split('-'))==2 else x)

In [6]:
pi[:1]

,cnum,pagination,reference,talk,journal,volume,year,doi,authors,title,pages,imprint
0,NaN,075008,"Phys. Rev. D 99, 075008 (2019)",NaN,Phys.Rev.,D99,2019,10.1103/PhysRevD.99.075008,"[Calle, Julian, Restrepo, Diego, Yaguna, Carlos E., Zapata, Óscar]",Minimal radiative Dirac neutrino mass models,9,2019-04-10


In [7]:
minyear=2017
pii=pi[pi['year']>=minyear].reset_index(drop=True)

In [8]:
pii.shape

(9, 12)

In [18]:
from IPython.display import clear_output

for i in pii.index:
    clear_output()
    print("="*80)
    print('Título del artículo (*)')
    print(pii.loc[i,'title'])
    print("\n")
    print( "Página Inicial(*)    Página Final(*)" )
    pages=pii.loc[i,'pagination'].split('-')
    if len(pages)==2:
        print( "{}                    {}".format(pages[0],pages[-1]))
    else:
        print( "{}                    {}".format(1,pii.loc[i,'pages']))
    print("\n")
    print("Año(*)      Mes")
    print('{}        {}'.format(pii.loc[i,'imprint'].split('-')[0],
                                       pii.loc[i,'imprint'].split('-')[1]))
    print("\n")
    print("Revista(*)")
    print(pii.loc[i,'journal'])
    print("\n")
    print("Volumen(*)    Fascículo Revista    Serie Revista")
    pagination=pii.loc[i,'pagination']
    if len(pages)==2:
        pagination=''
            
    print('{}           {}'.format( pii.loc[i,'volume'],
                                        pagination) )       
    print("\n")
    print("Sitio web (URL)       DOI (Digital Object Identifier)")
    print('{}           {}'.format( 'http://doi.org/'+pii.loc[i,'doi'],
                                        pii.loc[i,'doi'] ))
    print("\n")
    print("Autores")
    print('; '.join( pii.loc[i,'authors'] ))
    input('Hit <ENTER> to continue...')

Título del artículo (*)
Fermionic triplet dark matter in an $SO(10)$-inspired left right model


Página Inicial(*)    Página Final(*)
1                    9


Año(*)      Mes
2017        05


Revista(*)
Phys.Rev.


Volumen(*)    Fascículo Revista    Serie Revista
D95           095034


Sitio web (URL)       DOI (Digital Object Identifier)
http://doi.org/10.1103/PhysRevD.95.095034           10.1103/PhysRevD.95.095034


Autores
Arbeláez, Carolina; Hirsch, Martin; Restrepo, Diego
Hit <ENTER> to continue...


# ISSN from crossref api

In [366]:
#Default sort is descending sort=asc to change
tmp=pd.read_json( 'https://api.crossref.org/prefixes/10.1142/works?select=ISSN&sort=published').loc['items'][0]
kk=pd.DataFrame(tmp).dropna().sum()

In [367]:
pd.np.unique( kk.ISSN)

array(['0212-6567', '1578-1275'], dtype='<U9')

In [48]:
j={'Phys.Rev.':{'ISSN':['2470-0010','2470-0029'],'name':'Physical Review D'},
 'Phys.Lett.':{'ISSN':['0370-2693'],'name':'Physics Letters B'},
 'Mod.Phys.Lett.':{'ISSN':['0217-7323'],'name':'Modern Physics Letters A'},
 'Nucl.Phys.':{'ISSN':['0550-3213'],'name':'Nuclear Physics B'},
 'JHEP':{'ISSN':['1126-6708','1029-8479'],'name':'Journal of High Energy Physics'},
 'Int.J.Mod.Phys.':{'ISSN':['1793-656X'],'name':'International Journal Of Modern Physics A'},
 'JCAP':{'ISSN':['1475-7516'],'name':'Journal of Cosmology and Astroparticle Physics'},
 'Nucl.Part.Phys.Proc.':{'ISSN':[],'name':''},
   'J.Phys.':{'ISSN':[],'name':''},
   'Eur.Phys.J.':{'ISSN':[],'name':''},
   'Nucl.Phys.Proc.Suppl.':{'ISSN':[],'name':''},
   'Adv.High Energy Phys.':{'ISSN':[],'name':''},
   'Europhys.Lett.':{'ISSN':[],'name':''},
   'AIP Conf.Proc.':{'ISSN':[],'name':''},
   'Acta Phys.Hung.':{'ISSN':[],'name':''},
   'PoS':{'ISSN':[],'name':''}
}
pi['journal_details']=pi.journal.apply(lambda x: x if pd.isnull(x) else j.get(x) )

Listado revistas homologadas: https://drive.google.com/drive/folders/0B-mjxCw5cGbWWnRjcXA2bjBXZ2c?usp=sharing

Convert PDF to Excel:
https://www.ilovepdf.com

## Checked format

In [42]:
pi.columns

Index(['cnum', 'pagination', 'reference', 'talk', 'journal', 'volume', 'year',
       'doi', 'authors', 'title', 'pages', 'imprint'],
      dtype='object')

In [45]:
pi[['title','journal','authors','volume','pagination','pages','year','imprint','doi','journal_details']].sort_values('year',ascending=False)#.loc[4]

,title,journal,authors,volume,pagination,pages,year,imprint,doi,journal_details
0,Minimal radiative Dirac neutrino mass models,Phys.Rev.,"[Calle, Julian, Restrepo, Diego, Yaguna, Carlos E., Zapata, Óscar]",D99,075008,9,2019,2019-04-10,10.1103/PhysRevD.99.075008,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
2,Two-component dark matter and a massless neutrino in a new $B-L$ model,Phys.Rev.,"[Bernal, Nicolás, Restrepo, Diego, Yaguna, Carlos, Zapata, Óscar]",D99,015038,12,2019,2019-01-31,10.1103/PhysRevD.99.015038,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
3,Bound-state dark matter with Majorana neutrinos,Phys.Lett.,"[Reig, M., Restrepo, D., Valle, J.W. F., Zapata, O.]",B790,303-307,5,2019,2019-03-10,10.1016/j.physletb.2019.01.023,"{'ISSN': ['0370-2693'], 'name': 'Physics Letters B'}"
4,"Probing a simplified, $W^{\prime}$ model of $R(D^{(\ast)})$ anomalies using $b$-tags, $\tau$ leptons and missing energy",Phys.Rev.,"[Abdullah, Mohammad, Calle, Julian, Dutta, Bhaskar, Flórez, Andrés, Restrepo, Diego]",D98,055016,8,2018,2018-09-14,10.1103/PhysRevD.98.055016,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
5,Bound-state dark matter and Dirac neutrino masses,Phys.Rev.,"[Reig, M., Restrepo, D., Valle, J.W.F., Zapata, O.]",D97,115032,5,2018,2018-06-20,10.1103/PhysRevD.97.115032,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
7,Vector Boson Fusion in the Inert Doublet Model,Phys.Rev.,"[Dutta, Bhaskar, Palacio, Guillermo, Ruiz-Alvarez, Jose D., Restrepo, Diego]",D97,055045,8,2018,2018-03-29,10.1103/PhysRevD.97.055045,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
8,Expanding the Reach of Heavy Neutrino Searches at the LHC,Phys.Lett.,"[Flórez, Andrés, Gui, Kaiwen, Gurrola, Alfredo, Patiño, Carlos, Restrepo, Diego]",B778,94-100,7,2018,2018-03-10,10.1016/j.physletb.2018.01.009,"{'ISSN': ['0370-2693'], 'name': 'Physics Letters B'}"
9,"Anomalous leptonic U(1) symmetry: Syndetic origin of the QCD axion, weak-scale dark matter, and radiative neutrino mass",Mod.Phys.Lett.,"[Ma, Ernest, Restrepo, Diego, Zapata, Óscar]",A33,1850024,14,2018,2018-00-00,10.1142/S0217732318500244,"{'ISSN': ['0217-7323'], 'name': 'Modern Physics Letters A'}"
10,Fermionic triplet dark matter in an $SO(10)$-inspired left right model,Phys.Rev.,"[Arbeláez, Carolina, Hirsch, Martin, Restrepo, Diego]",D95,095034,9,2017,2017-05-31,10.1103/PhysRevD.95.095034,"{'ISSN': ['2470-0010', '2470-0029'], 'name': 'Physical Review D'}"
15,The Inert Zee Model,JHEP,"[Longas, Robinson, Portillo, Dilia, Restrepo, Diego, Zapata, Oscar]",1603,162,12,2016,2016-03-23,10.1007/JHEP03(2016)162,"{'ISSN': ['1126-6708', '1029-8479'], 'name': 'Journal of High Energy Physics'}"
